In [ ]:
#Imports
from math import exp, cos, sqrt, inf, pi, floor
%pip install numpy
import numpy as np
%pip install tqdm
from tqdm import tqdm
%pip install matplotlib
%matplotlib inline
import matplotlib.pyplot
import matplotlib.pyplot as plt
import copy
import random
from IPython import display
import pylab as pl
from matplotlib.patches import Ellipse
%pip install scikit-learn
from sklearn import datasets
import numpy as np
import time
from sklearn import datasets

plt.rcParams["font.family"]='Times New Roman'
plt.rcParams["mathtext.fontset"]='stix'

from IPython.core.display import display, HTML, Math, Latex
display(HTML("<style>.rendered_html { font-size: 16px; }</style>"))

In [ ]:
#Test Functions, x is a list of all x_i
rastrigin = lambda x : 10*len(x) + sum((x_i**2 - 10*cos(2*pi*x_i)) for x_i in x)
def rosenbrock_function(x):
    sum = 0
    for i in range(len(x)-1):
      sum += 100 * (x[i+1] - x[i] ** 2) ** 2 + (x[i] - 1) ** 2
    return sum
rosenbrock = lambda x : rosenbrock_function(x)
styblinski = lambda x : 0.5*sum((x_i**4 - 16*x_i**2 + 5*x_i) for x_i in x)

In [ ]:
#Initiation Functions - Both functions would return an array of a population with a chromosome consisting of n dimensions
#Make the population size even (it makes it easier down the line)

#Uniform Random Initialization
def uniform_random_init(pop_size:int, n:int, params:list)->list:
    x_i_min = params[0]
    x_i_max = params[1]
    population = []
    for i in range(pop_size):
        chromosome = np.random.rand(n) * (x_i_max-x_i_min) + x_i_min
        population.append(chromosome)
    return population

#Normal Distribution Initialization
def normal_distribution_init(pop_size:int, n:int, params:list)->list:
    mean = params[0]
    stddev = params[1]
    population = []
    for i in range(pop_size):
        chromosome = np.random.normal(loc=mean, scale=stddev, size=(n))
        population.append(chromosome)
    return population

In [ ]:
#Selection Functions - Both functions will return an array that will list n indexes from the population array that will be used for sexual crossover
#Both functions support elitism

#Tournament Selection
def tournament_selection(pop:list, fitness, t:int, params:list, elitism = True, high_pc = False)->list:
    n = params[0]
    n_ts = params[1]
    fit_score_pop = [] #Array with a set of (index from population, fitness score)
    selection_pop = [] #Array containing the indexes of all from the population that will be used for crossover
    if elitism:
        best_idx = 0
        best_fit = inf

    #Fill fit_score_pop and find the best if elitism is enabled
    for creature in enumerate(pop):
        fit_score = fitness(creature[1])
        fit_score_pop.append([creature[0], fit_score])
        if elitism:
            if best_fit > fit_score:
                best_idx = creature[0]
                best_fit = fit_score

    if elitism:
        selection_pop.append(fit_score_pop[best_idx][0])

    if high_pc:
        idx_in_list = [best_idx]
        while(len(selection_pop) < len(pop)/2):
            idx = np.random.randint(len(pop))
            while idx in idx_in_list:
                idx = np.random.randint(len(pop))
            selection_pop.append(fit_score_pop[idx][0])
            idx_in_list.append(idx)

    #Tournament selection process
    while(len(selection_pop)<n):
        tournament_pop = random.sample(fit_score_pop, n_ts)

        best_idx = 0
        best_fit = inf
        for participant in tournament_pop:
            if(best_fit > fit_score):
                best_idx = participant[0]
                best_fit = participant[1]

        selection_pop.append(fit_score_pop[best_idx][0])

    return selection_pop

#Boltzmann Selection
def boltzmann_selection(pop:list, fitness, t:int, params:list, elitism = True, high_pc = False)->list:
    n = params[0]
    T_func = params[1]
    correction_factor = 10**35

    fit_score_pop = [] #Array with a set of (index from population, fitness score)
    selection_pop = [] #Array containing the indexes of all from the population that will be used for crossover

    best_idx = 0
    best_fit = inf

    #Fill fit_score_pop and find the best if elitism is enabled
    for creature in enumerate(pop):
        fit_score = fitness(creature[1])
        fit_score_pop.append([creature[0], fit_score])
        if best_fit > fit_score:
            best_idx = creature[0]
            best_fit = fit_score

    if elitism:
        selection_pop.append(fit_score_pop[best_idx][0])

    if high_pc:
        idx_in_list = [best_idx]
        while(len(selection_pop) < len(pop)/2):
            idx = np.random.randint(len(pop))
            while idx in idx_in_list:
                idx = np.random.randint(len(pop))
            selection_pop.append(fit_score_pop[idx][0])
            idx_in_list.append(idx)

    #Boltzmann distribution
    demoninator = 0.0
    T = T_func(t)
    for creature in fit_score_pop:
        demoninator += exp(-(creature[1]/correction_factor))/(T)

    rand_percent = np.random.rand()
    running_total_percent = 0.0
    while(len(selection_pop)<len(pop)/2):
        for creature in fit_score_pop:
            running_total_percent += exp(-(creature[1]/correction_factor)/T)/demoninator

            if(running_total_percent >= rand_percent):
                selection_pop.append(creature[0])
                running_total_percent = 0
                rand_percent = np.random.rand()

    return selection_pop


In [ ]:
#Crossover Operators - given the general population and the selection population from the selection functions, return some children

#Two Point - choose two parents randomly, return a list of two children
def two_point_cross(pop:list, selection_pop:list, n_dim:int, params=None) -> list:
    parent_1 = pop[selection_pop[np.random.randint(low = 0, high=len(selection_pop))]]
    parent_2 = pop[selection_pop[np.random.randint(low = 0, high=len(selection_pop))]]
    point_1 = np.random.randint(low=0, high=n_dim)
    point_2 = np.random.randint(low=0, high=n_dim)

    #ensure that point 1 is less than point 2
    if point_1 < point_2:
        point_2, point_1 = point_1, point_2
    elif point_1 == point_2:
        point_2 += 1

    child_1 = parent_1.copy()
    child_2 = parent_2.copy()

    child_1[point_1:point_2] = parent_2[point_1:point_2]
    child_2[point_1:point_2] = parent_1[point_1:point_2]

    return [child_1, child_2]


#UNDX - choose 3 parents randomly, return m children
def undx_cross(pop:list, selection_pop:list,  n_dim:int, params:list) -> list:
    m = params[0]
    sigma_1 = params[1]
    sigma_2 = params[2]

    parents = random.sample(selection_pop, 3)

    #find midpoint of the first two parents
    midpoint = np.zeros(n_dim)

    for dim in range(n_dim):
        for parent_idx in parents[:-1]:
            midpoint[dim] += pop[parent_idx][dim]
        midpoint[dim] = midpoint[dim]/2

    #create the vector between 1 and 2 and the vector between 1 and 3
    v_12 = np.zeros(n_dim)
    v_13 = np.zeros(n_dim)

    for dim in range(n_dim):
        v_12[dim] = pop[parents[1]][dim] - pop[parents[0]][dim]
        v_13[dim] = pop[parents[2]][dim] - pop[parents[0]][dim]

    #Get the difference vector (from parent 2 to parent 1)
    v_diff = -1.0 * v_12

    #Find distance from 3rd parent to primary search line
    e_12 = v_12 / np.clip(np.linalg.norm(v_12, ord=2), 1e-10, None)
    v_12_3 = v_13 - np.dot(v_13, e_12) * e_12 #Vector orthogonal to search line through 3
    distance = np.linalg.norm(v_12_3, ord=2)

    #Find orthogonal basis vector for the subspace that are orthogonal to the search line
    basis_matrix = np.identity(n_dim)

    if(np.count_nonzero(e_12) != 0):
        basis_matrix[0] = e_12

    basis_matrix_t = basis_matrix.T
    Q, _ = np.linalg.qr(basis_matrix_t)

    basis_vectors = Q.T[1:]

    #create children
    children = []

    for i in range(m):
        child = np.zeros(n_dim)
        norm_1 = np.random.normal(0, sigma_1)
        norm_2 = np.random.normal(0, sigma_2, size = n_dim)
        
        child = midpoint + norm_1 * v_diff

        #Getting the third term
        three = np.zeros(n_dim)
        for i in range(n_dim-1):
            three += norm_2[i] * basis_vectors[i]
        three *= distance

        child += three

        children.append(child)

    return children

#Main Crossover Function - Returns the new population (size len(pop)) after selection
def crossover(function, elitism:bool, pop:list, selection_pop:list,  n_dim:int, params:list, high_pc = False):
    new_pop = []
    if(elitism):
        new_pop.append(pop[selection_pop[0]])

    if high_pc:
        for i in range(floor(len(pop)/2) - 1):
            new_pop.append(pop[selection_pop[i+1]])

    while(len(new_pop)<len(pop)):
        new_children = function(pop, selection_pop, n_dim, params)
        for children in new_children:
            if(len(new_pop)<len(pop)): #make sure that no additional children are added
                new_pop.append(children)

    return new_pop


In [ ]:
#Mutation Operators - given a chromosome, return a mutated chromosome

#Normal Distributon Pertution Mutation
def normal_dis_mutation(chromosome:list, mutate_chance:float, params:list) -> list:
    sigma = params[0]
    for gene in range(len(chromosome)):
        mutate = random.uniform(0, 1)
        if mutate<=mutate_chance:
            chromosome[gene] += np.random.normal(0, sigma)
    return chromosome

#Swap Mutation
def swap_mutation(chromosome:list, mutate_chance:float, params=None) -> list:
    for gene_idx in range(len(chromosome)):
        mutate = random.uniform(0, 1)
        if mutate<=mutate_chance:
            swap_idx = np.random.randint(0, high=len(chromosome))
            temp = chromosome[gene_idx].copy()
            chromosome[gene_idx] = chromosome[swap_idx].copy()
            chromosome[swap_idx] = temp
    return chromosome

#Main Mutation Function - returns a list of the new generation
def mutate(function, elitism:bool, crossover_pop:list, mutate_chance, params:list, high_pc = False) -> list:
    i = 0
    new_pop = []
    for creature in crossover_pop:
        if i==0 and elitism:
            new_pop.append(creature)
            i+=1
            continue
        if high_pc and i<50:
            new_pop.append(creature)
            i+=1
            continue
        new_pop.append(function(creature, mutate_chance, params))
    return new_pop



In [ ]:
#Termination criteria - used in the loop to determine how it ends
#Returns true if the termination condition meets

#Max Amount of Generations Termination
def enough_time_termination(t_current:int, t_max:int):
    return t_current>=t_max

#Slow Changing Max Termination
t_since_change = 0.0
starting_max = 0.0
def slow_changing_max_termination(fit_max:float, t_max = 100):
    global starting_max
    global t_since_change
    if(fit_max == starting_max):
        t_since_change += 1
        return t_since_change > t_max
    else:
        starting_max = fit_max
        t_since_change = 0
        return False


In [ ]:
#Defining parameters for each function/operator
init_func = [[uniform_random_init, [-5.12, 5.12]], [normal_distribution_init, [0,2]]]
selection_func = [[tournament_selection, [200*100, 3]], [boltzmann_selection, [200*100, lambda t : 1000*exp(-t/100)]]]
crossover_func = [[two_point_cross, None], [undx_cross, [3, 0.5, 0.35/sqrt(2)]]]
mutation_func = [[normal_dis_mutation, [10.24/12]], [swap_mutation, None]]

In [ ]:
#Find the fitness for a whole population, sorted by the score function
def fitness_pop(pop:list, fitness_function, fitness_min):
    scores = []
    for chromosome in pop:
        fitness = fitness_function(chromosome)
        scores.append( 1.0 / round((fitness - fitness_min + 1),100))
    scores, sorted_pop = np.array(scores), np.array(pop.copy())
    inds = np.argsort(scores)
    scores = list(scores[inds][::-1])
    sorted_pop = list(sorted_pop[inds,:][::-1])
    return scores, sorted_pop

In [ ]:
#Main Program Loop
from tqdm import tqdm_notebook as tqdm

def generations(sz, elitism, mutation_rate, n_dim, n_gen_max, fitness_func, fitness_min, init, selection, cross, mutation, terminate, high_pc = False):
    best_chromo = []
    best_score = []
    population_nextgen = init_func[init][0](sz,n_dim,init_func[init][1])
    stats_min = np.zeros(n_gen_max)
    stats_avg = np.zeros(n_gen_max)
    stats_max = np.zeros(n_gen_max)
    stats_std = np.zeros(n_gen_max)
    #Store the points from generation to generation
    stored_points = []
    # this is the number of generations
    current_generation = 0

    #Get initial population
    _, pop_after_fit = fitness_pop(population_nextgen, fitness_func, fitness_min)
    stored_points.append( pop_after_fit )

    while ( (terminate==0 and not enough_time_termination(current_generation, 1000))                #Termination Condition 1
            or (terminate==1 and not slow_changing_max_termination(stats_max[current_generation-1]))  #Termination Condition 2
            and (current_generation < n_gen_max)):                                                  #This is taking too long
        # evlauate & select
        pop_sel = selection_func[selection][0](population_nextgen,fitness_func, current_generation, selection_func[selection][1], elitism, high_pc)
        # crossover
        pop_after_cross = crossover(crossover_func[cross][0],elitism, population_nextgen, pop_sel, n_dim, crossover_func[cross][1], high_pc)
        # mutation
        population_nextgen = mutate(mutation_func[mutation][0], elitism, pop_after_cross, mutation_rate, mutation_func[mutation][1], high_pc)
        # I do one more scoring as I track values from epoch to epoch (could just change where its at also ;-)
        scores, pop_after_fit = fitness_pop(population_nextgen, fitness_func, fitness_min)
        stored_points.append( pop_after_fit ) # remember each epoch
        best_chromo.append(pop_after_fit[0].copy())
        best_score.append(scores[0].copy())
        stats_min[current_generation] = np.min(scores)
        stats_max[current_generation] = np.amax(scores)
        stats_avg[current_generation] = np.mean(scores)
        stats_std[current_generation] = np.std(scores)

        current_generation += 1

    #Reset Termination Condition 2
    global t_since_change
    global starting_max
    t_since_change = 0.0
    starting_max = 0.0
    
    return best_chromo,best_score,stats_min,stats_avg,stats_max,stats_std,stored_points, current_generation

In [ ]:
s = 100 # number of individuals
ngen = 10000 # maximum number of generations
mutation_rate_low = 0.05
mutation_rate_high = 0.60

In [ ]:
def ras_2d(x, y):
    z = 20 + x*x - 20*np.cos(2*pi*x) + y*y - 20*np.cos(2*pi*y)
    return z

def rosen_2d(x,y):
    z = 100.0*(y-x*x)*(y-x*x) + (1-x)*(1-x)
    return z

def styb_2d(x,y):
    z = 0.5*((x*x*x*x- 16*x*x + 5*x) + (y*y*y*y- 16*y*y + 5*y))
    return z

def print_stats(stats_min, stats_avg, stats_max, stats_std, executed_gen):
    print(f"Number of generations: {executed_gen}")
    print(f"Min: {round(stats_min[executed_gen-1],8)},")
    print(f"Avg: {round(stats_avg[executed_gen-1],8)}")
    print(f"Max: {round(stats_max[executed_gen-1],8)}")
    print(f"Std Deviation: {round(stats_std[executed_gen-1],8)}")

def graph(stats_min, stats_avg, stats_max, executed_gen, title):
    plt.plot(stats_min[:executed_gen],'r')
    plt.plot(stats_avg[:executed_gen],'b')
    plt.plot(stats_max[:executed_gen],'g')
    plt.ylabel('Accuracy')
    plt.xlabel('Generations')
    plt.title(title)
    plt.show()

def contour(stored_points, fit_array_func):
    fig, ax = pl.subplots(nrows = 1, ncols = 1, figsize=(7, 5))
    for t in range(len(stored_points)):
        pl.clf()

        # get this set of points out
        data = stored_points[t]

        # plot
        for i in range(len(data)):
            pl.scatter(data[i][0], data[i][1], edgecolor='b', alpha=0.3)

        a = np.arange(-5.14, 5.14, 0.05)
        b = np.arange(-5.14, 5.14, 0.05)
        x, y = np.meshgrid(a, b)
        z = fit_array_func(x,y)
        pl.contour(x, y, z, levels=np.logspace(-9, 9, 50), cmap='jet', alpha=0.4)

        display.clear_output(wait=True)
        display.display(pl.gcf())
        plt.xlim(-5.14, 5.14)
        plt.ylim(-5.14, 5.14)
        time.sleep(0.01)

def contour_one_gen(stored_points, fit_array_func, title):
    fig, ax = pl.subplots(nrows = 1, ncols = 1, figsize=(7, 5))
    data_x = []
    data_y = []
    for i in range(len(stored_points)):
        data_x.append(stored_points[i][0])
        data_y.append(stored_points[i][1])
    
    # plot
    colors = np.random.rand(len(stored_points))
    plt.scatter(data_x, data_y, c=colors, alpha=0.5)

    a = np.arange(-5.14, 5.14, 0.05)
    b = np.arange(-5.14, 5.14, 0.05)
    x, y = np.meshgrid(a, b)
    z = fit_array_func(x,y)

    pl.contour(x, y, z, levels=np.logspace(-9, 9, 50), cmap='jet', alpha=0.4)

    plt.xlim(-5.14, 5.14)
    plt.ylim(-5.14, 5.14)
    plt.title(title)
    plt.show()

In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
# Everything below this point is a mess, but this is how I tested my functions
# Many things change between trials, the following is not a static program
#
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
for i in range(16): 
    bit_mask = '{0:05b}'.format(i)
    if(bit_mask[4]=="1" or bit_mask[2]!="1"):
        continue
    chromo,score,stats_min,stats_avg,stats_max,stats_std, stored_points, executed_gen = generations(s, True, mutation_rate_high, 2, ngen, rastrigin, 0, 
                                                                                                    int(bit_mask[0]), int(bit_mask[1]), 
                                                                                                    int(bit_mask[2]), int(bit_mask[3]), 
                                                                                                    int(bit_mask[4]), high_pc = True)
    print(f"Stats for GA {bit_mask} = {i}")
    print_stats(stats_min, stats_avg, stats_max, stats_std, executed_gen)
    graph(stats_min, stats_avg, stats_max, executed_gen, f"GA {bit_mask}")
    #contour_one_gen(stored_points[0], ras_2d, f"Population for GA {bit_mask} at Generation 1")
    #contour_one_gen(stored_points[executed_gen-1], ras_2d, f"Population for GA {bit_mask} at Generation {executed_gen}")
    #contour_one_gen(stored_points[floor((executed_gen)/2)-1], ras_2d, f"Population for GA {bit_mask} at Generation {floor((executed_gen)/2)}")

In [ ]:
for i in range(16): 
    bit_mask = '{0:05b}'.format(i)
    if (bit_mask[4] == "1"):
        continue
    chromo,score,stats_min,stats_avg,stats_max,stats_std, stored_points, executed_gen = generations(s, True, mutation_rate_high, 2, ngen, rosenbrock, 0, 
                                                                                                    int(bit_mask[0]), int(bit_mask[1]), 
                                                                                                    int(bit_mask[2]), int(bit_mask[3]), 
                                                                                                    int(bit_mask[4]), high_pc=True)
    print(f"Stats for GA {bit_mask}")
    print_stats(stats_min, stats_avg, stats_max, stats_std, executed_gen)
    graph(stats_min, stats_avg, stats_max, executed_gen, f"GA {bit_mask}")
    #contour_one_gen(stored_points[0], rosen_2d, f"Population for GA {bit_mask} at Generation 1")
    #contour_one_gen(stored_points[executed_gen-1], rosen_2d, f"Population for GA {bit_mask} at Generation {executed_gen}")
    #contour_one_gen(stored_points[floor((executed_gen)/2)-1], rosen_2d, f"Population for GA {bit_mask} at Generation {floor((executed_gen)/2)}")

In [ ]:
for i in range(16): 
    bit_mask = '{0:05b}'.format(i)
    if (bit_mask[4] == "1"):
        continue
    chromo,score,stats_min,stats_avg,stats_max,stats_std, stored_points, executed_gen = generations(s, True, mutation_rate_high, 2, ngen, 
                                                                                                    styblinski, -39.16617*2, 
                                                                                                    int(bit_mask[0]), int(bit_mask[1]), 
                                                                                                    int(bit_mask[2]), int(bit_mask[3]), 
                                                                                                    int(bit_mask[4]), high_pc=True)
    print(f"Stats for GA {bit_mask}")
    print_stats(stats_min, stats_avg, stats_max, stats_std, executed_gen)
    graph(stats_min, stats_avg, stats_max, executed_gen, f"GA {bit_mask}")
    #contour_one_gen(stored_points[0], styb_2d, f"Population for GA {bit_mask} at Generation 1")
    #contour_one_gen(stored_points[executed_gen-1], styb_2d, f"Population for GA {bit_mask} at Generation {executed_gen}")
    #contour_one_gen(stored_points[floor((executed_gen)/2)-1], styb_2d, f"Population for GA {bit_mask} at Generation {floor((executed_gen)/2)}")